# Example workflow for using flosp to import and clean ED data

Get to directory above to import flosp

In [1]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is EE63-178E

 Directory of C:\Users\bjk1y13\OneDrive - University of Southampton\MH030_HHFT_flow\4_Analysis\flosp\example

05/02/2019  14:09    <DIR>          .
05/02/2019  14:09    <DIR>          ..
04/01/2019  11:00    <DIR>          .ipynb_checkpoints
05/02/2019  13:23    <DIR>          __pycache__
09/01/2019  14:59         3,512,358 example_data_ED.csv
05/02/2019  12:12        20,308,140 example_data_IP.csv
05/02/2019  14:09            51,291 example_flosp_workflow.ipynb
05/02/2019  10:13    <DIR>          example_results
05/02/2019  13:19               214 setup.py
               4 File(s)     23,872,003 bytes
               5 Dir(s)  337,669,431,296 bytes free


In [2]:
%cd ".\..

C:\Users\bjk1y13\OneDrive - University of Southampton\MH030_HHFT_flow\4_Analysis\flosp


In [3]:
import flosp

## initialise flosp

You must provide the path to the setup file, which you should edit to your project specifics.

In [4]:
from flosp import Interface

In [5]:
hosp = Interface(setup_file_path='./setup.py')

flosp started.
<module 'flosp.expected_structures.ED' from 'C:\\Users\\bjk1y13\\OneDrive - University of Southampton\\MH030_HHFT_flow\\4_Analysis\\flosp\\flosp\\expected_structures\\ED.py'>
<module 'flosp.expected_structures.IP' from 'C:\\Users\\bjk1y13\\OneDrive - University of Southampton\\MH030_HHFT_flow\\4_Analysis\\flosp\\flosp\\expected_structures\\IP.py'>
Imported setup.py sucessfully.
Loading processed data from pickles.
Found: ./example/example_results/processed/ED.pkl
Missing: ./example/example_results/processed/IPfce.pkl
Missing: ./example/example_results/processed/IPward.pkl
Missing: ./example/example_results/processed/IPspell.pkl
Missing: ./example/example_results/processed/HOURLY.pkl
Missing: ./example/example_results/processed/DAILY.pkl


### Importing ED data

load raw data

In [6]:
hosp.load_dataED('./example/example_data_ED.csv') #,nrows=1500) # limit number of rows for quick runtime during dev

IO class called.
ARRIVAL_DTTM
TRIAGE_ASSESSMENT_DTTM
FIRSTDOC_FOR_TREATMENT_DTTM
ADM_REQUEST_DTTM
ATTENDANCE_CONCLUSION_DTTM
----------------------------------------
Making callender columns from:  ARRIVAL_DTTM
----------------------------------------
Making callender columns from:  ATTENDANCE_CONCLUSION_DTTM
----------------------------------------
saved file: ./example/example_results/processed/ED.pkl
Import completed.


look at the dataframe at any stage of processing

In [7]:
hosp.extract_data('ED').head()

,PSEUDONYMISED_PATIENT_ID,AGE_AT_ARRIVAL,GENDER_NATIONAL_DESCRIPTION,SITE,ARRIVAL_DTTM,ARRIVAL_MODE_NATIONAL_CODE,TRIAGE_ASSESSMENT_DTTM,FIRSTDOC_FOR_TREATMENT_DTTM,ADM_REQUEST_DTTM,ADM_REQUEST_LOC_DESCRIPTION,...,age_group,waiting_time,arr_triage_wait,arr_dr_wait,arr_adm_req_wait,adm_req_dep_wait,dr_adm_req_wait,dr_dep_wait,breach_flag,breach_datetime
0,6473,86,Male,HOSPITAL1,2015-12-01 04:49:00,1.0,2015-11-29 03:40:00,2015-11-30 04:19:00,2015-11-30 06:08:53,General Medicine,...,65+,642.0,-2949.0,-1470.0,-1360.116667,2002.116667,109.883333,2112.0,1,2015-12-01 08:49:00
1,3955,20,Male,HOSPITAL1,2016-06-23 13:41:00,2.0,2016-06-23 14:47:00,2016-06-24 16:58:00,NaT,NaN,...,18-64,208.0,66.0,1637.0,NaN,NaN,NaN,-1429.0,0,2016-06-23 17:41:00
2,2880,127,Male,HOSPITAL1,2015-08-15 20:03:00,1.0,2015-08-15 20:23:00,2015-08-13 22:08:00,NaT,NaN,...,65+,176.0,20.0,-2755.0,NaN,NaN,NaN,2931.0,0,2015-08-16 00:03:00
3,5278,11,Female,HOSPITAL1,2016-07-26 17:39:00,2.0,2016-07-24 18:14:00,2016-07-24 19:23:00,NaT,NaN,...,0-17,148.0,-2845.0,-2776.0,NaN,NaN,NaN,2924.0,0,2016-07-26 21:39:00
4,1950,83,Male,HOSPITAL1,2016-05-23 09:04:00,1.0,2016-05-24 08:09:00,2016-05-21 08:17:00,2016-05-23 09:53:40,General Medicine,...,65+,1634.0,1385.0,-2927.0,49.666667,1584.333333,2976.666667,4561.0,1,2016-05-23 13:04:00


subsampling method - for quicker inital runtime during development of cleaning (some of the datetime conversions can take some time on larger data sets

## Import IP data

In [8]:
hosp.load_dataIP('./example/example_data_IP.csv')

IO class called.
ADM_DTTM
DIS_DTTM
MOVE_START
MOVE_END
----------------------------------------
Making callender columns from:  ADM_DTTM
----------------------------------------
Making callender columns from:  DIS_DTTM


C:\Users\bjk1y13\OneDrive - University of Southampton\MH030_HHFT_flow\4_Analysis\flosp\flosp\basic_tools.py:99: UserWarning: Some datetimes in your column are missing.
  warnings.warn('Some datetimes in your column are missing.')


----------------------------------------
saved file: ./example/example_results/processed/IP.pkl
Import completed.


In [12]:
hosp.extract_data('IP').head()

,MOVE_NUMBER,ADM_SOURCE_CODE,ADM_METHOD,ADM_DTTM,ADM_TYPE,SITE,GENDER_NATIONAL_DESCRIPTION,MOVE_END,DIS_DESTINATION,DIS_METHOD,...,DIS_hour,DIS_dayofweek,DIS_day,DIS_month,DIS_week,DIS_dayofweek_name,DIS_year,DIS_date,DIS_flag_wkend,age_group
0,1,19,31,2017-07-05 07:32:00,Non-Elective,HOSPITAL1,2,2017-07-08 13:53:00,19.0,1.0,...,14.0,5.0,8.0,7.0,27.0,Saturday,2017.0,2017-07-08,0,18-64
1,1,79,82,2017-06-17 05:03:00,Day Case,HOSPITAL1,1,2017-06-20 22:41:00,19.0,1.0,...,21.0,1.0,20.0,6.0,25.0,Tuesday,2017.0,2017-06-20,0,18-64
2,9,19,21,2017-11-09 22:08:00,Non-Elective,HOSPITAL1,1,2017-12-19 09:38:00,19.0,1.0,...,15.0,2.0,20.0,12.0,51.0,Wednesday,2017.0,2017-12-20,0,65+
3,12,19,21,2017-12-08 17:41:00,Non-Elective,HOSPITAL1,1,2018-02-03 11:00:00,19.0,1.0,...,11.0,5.0,3.0,2.0,5.0,Saturday,2018.0,2018-02-03,0,65+
4,2,19,21,2018-04-07 15:25:00,Non-Elective,HOSPITAL1,1,2018-04-11 19:51:00,19.0,1.0,...,19.0,2.0,11.0,4.0,15.0,Wednesday,2018.0,2018-04-11,0,65+


In [9]:
break

SyntaxError: 'break' outside loop (<ipython-input-9-6aaf1f276005>, line 1)

### mapping columns to standard naming convention

unless your columns are already named using the required names - use column mapping method

In [1]:
#### define dictionary for mapping
col_map = {
'PSEUDONYMISED_PATIENT_ID':'dept_patid',
'PSEUDONYMISED_PATIENT_ID':'hosp_patid',
'AGE_AT_ARRIVAL':'age',
'GENDER_NATIONAL_DESCRIPTION':'gender',
'SITE':'site',
'ARRIVAL_DTTM':'arrive_datetime',
'ARRIVAL_MODE_NATIONAL_CODE':'arrive_mode',
'INITIAL_ASSESSMENT_DTTM':'first_triage_datetime',
'SEEN_FOR_TREATMENT_DTTM':'first_dr_datetime',
'SPECIALTY_REQUEST_TIME':'first_adm_request_time',
'SPECIALTY_REFERRED_TO_DESCRIPTION':'adm_referral_loc',
'ADMISSION_FLAG':'adm_flag',
'ATTENDANCE_CONCLUSION_DTTM':'depart_datetime',
'STREAM_LOCAL_CODE':'stream'
}

In [3]:
len(col_map)

13

In [7]:
EDdata.map_columns(col_map)

----------------------------------------
mapping column names


required column names can be found: 

In [8]:
flosp._expected_file_structures.dataRAW_expected_cols.keys()

dict_keys(['hosp_patid', 'age', 'age_group', 'gender', 'arrive_datetime', 'arrive_mode', 'arrive_hour', 'arrive_dayofweek', 'arrive_month', 'arrive_dayofweek_name', 'arrive_date', 'arrive_week', 'first_triage_datetime', 'first_dr_datetime', 'first_adm_request_datetime', 'adm_referral_loc', 'depart_datetime', 'depart_method', 'depart_hour', 'depart_dayofweek', 'depart_week', 'depart_month', 'depart_dayofweek_name', 'depart_date', 'waiting_time', 'breach_flag', 'adm_flag', 'stream', 'minutes_today', 'minutes_tomo', 'breach_datetime', 'arr_triage_wait', 'arr_dr_wait', 'arr_adm_req_wait', 'adm_req_dep_wait', 'dr_adm_req_wait', 'dr_dep_wait'])

In [9]:
EDdata.get_EDraw().head(3)

,hosp_patid,age,gender,site,arrive_datetime,arrive_mode,first_triage_datetime,first_dr_datetime,first_adm_request_time,adm_referral_loc,adm_flag,depart_datetime,stream
0,2493,50,Female,HOSPITAL1,2013-01-08 19:44:00,2.0,2013-01-07 18:50:00,2013-01-09 21:50:00,NaN,NaN,0,2013-01-10 23:16:00,MIN
1,4822,22,Male,HOSPITAL1,2013-07-03 09:34:00,2.0,2013-07-03 08:57:00,2013-07-02 09:17:00,NaN,NaN,0,2013-07-05 09:02:00,MIN
2,6012,51,Male,HOSPITAL1,2017-08-27 23:33:00,2.0,2017-08-26 01:32:00,2017-08-27 03:12:00,NaN,NaN,0,2017-08-26 03:31:00,MIN


### bespoke user cleaning operations 

if at any stage manual edits to the data are required (e.g. cleaning a spurious datetime)... get data out of class, edit and replace:

In [10]:
#### get data out
df = EDdata.get_EDraw()

#### make changes to df
# my changes here

#### replace data into EDdata
EDdata.replace_EDraw(df)

### converting datetimes
#### datetime formats are often non-standard...some attention is needed but there are some built in methods to help.

convert columns to datetime formats (by default anything with 'datetime' in column name will be transformed.

In [11]:
EDdata.convert_cols_datetime("%Y/%m/%d %H:%M")

C:\Users\bjk1y13\OneDrive - University of Southampton\MH000_HHFT_flow\4_Analysis\flosp\flosp\ioED.py:101: UserWarning: Datetime conversion can be problemtic. Make sure you have used the correct datetime string format for each column. You can call this method multiple times with different 'datetime formats' & 'list of columns' to convert if neccessary.
  warnings.warn("Datetime conversion can be problemtic. Make sure you have used the correct datetime string format for each column. You can call this method multiple times with different 'datetime formats' & 'list of columns' to convert if neccessary.")


Converting cols to datetime...(may take some time depedning on size of df)...
arrive_datetime...converting
first_triage_datetime...converting
first_dr_datetime...converting
depart_datetime...converting


create a datetime column from seperate time and date columns.

In [12]:
EDdata.create_datetime_from_time('first_adm_request_time','arrive_datetime','first_adm_request_datetime')

----------------------------------------
Create datetime column from: first_adm_request_time & arrive_datetime


C:\Users\bjk1y13\OneDrive - University of Southampton\MH000_HHFT_flow\4_Analysis\flosp\flosp\basic_tools.py:131: UserWarning: 422 patients detected who have -ve wait times. They have probably rolled over midnight, so we add + 1day to the new datetime column created. This assumes < 24hr stays only.



  warnings.warn(str(datetime_values.shape[0]) + ' patients detected who have -ve wait times. They have probably rolled over midnight, so we add + 1day to the new datetime column created. This assumes < 24hr stays only.\n\n\n')


### automated cleaning

check what else needs doing to get data into normalised format:

In [13]:
EDdata.run_tests()

--------------------
Finding missing columns...
age_group try using:  use make_age_group_column
arrive_hour try using:  make_callender_columns
arrive_dayofweek try using:  make_callender_columns
arrive_month try using:  make_callender_columns
arrive_dayofweek_name try using:  make_callender_columns
arrive_date try using:  make_callender_columns
arrive_week try using:  make_callender_columns
depart_method try using:  
depart_hour try using:  make_callender_columns
depart_dayofweek try using:  make_callender_columns
depart_week try using:  make_callender_columns
depart_month try using:  make_callender_columns
depart_dayofweek_name try using:  make_callender_columns
depart_date try using:  make_callender_columns
waiting_time try using:  
breach_flag try using:  
minutes_today try using:  
minutes_tomo try using:  
breach_datetime try using:  
arr_triage_wait try using:  
arr_dr_wait try using:  
arr_adm_req_wait try using:  
adm_req_dep_wait try using:  
dr_adm_req_wait try using:  
dr_de

#### run as much automated cleaning as possible using: 

In [15]:
EDdata.autoclean()

----------------------------------------
Making callender columns from:arrive_datetime
----------------------------------------
Making callender columns from:depart_datetime


run tests again:

In [16]:
EDdata.run_tests()

--------------------
Finding missing columns...
depart_method try using:  
minutes_today try using:  
minutes_tomo try using:  
--------------------
Finding columns with wrong datatypes...
Col  arrive_mode                is: float64 . Expected any of:  [<class 'object'>]
Col  adm_flag                   is: int64 . Expected any of:  [<class 'int'>]
Col  stream                     is: object . Expected any of:  [<class 'float'>, <class 'int'>, <class 'numpy.int64'>, <class 'str'>]
Col  depart_dayofweek_name      is: object . Expected any of:  [<class 'str'>]
Col  age_group                  is: category . Expected any of:  [<class 'str'>, 'pandas category type']


#### alternativly to autoclean,  step through the process using various methods:

In [17]:
EDdata.make_callender_columns()

----------------------------------------
Making callender columns from:arrive_datetime
----------------------------------------
Making callender columns from:depart_datetime


In [18]:
EDdata.make_wait_columns()

In [19]:
EDdata.make_breach_columns()

In [20]:
EDdata.make_age_group_column()

In [21]:
EDdata.run_tests()

--------------------
Finding missing columns...
depart_method try using:  
minutes_today try using:  
minutes_tomo try using:  
--------------------
Finding columns with wrong datatypes...
Col  arrive_mode                is: float64 . Expected any of:  [<class 'object'>]
Col  adm_flag                   is: int64 . Expected any of:  [<class 'int'>]
Col  stream                     is: object . Expected any of:  [<class 'float'>, <class 'int'>, <class 'numpy.int64'>, <class 'str'>]
Col  depart_dayofweek_name      is: object . Expected any of:  [<class 'str'>]
Col  age_group                  is: category . Expected any of:  [<class 'str'>, 'pandas category type']


## saving
#### ioED (and flosp) will automate the structure of your saving folder in the root dir you provided when you created the EDdata instance of ioED

In [26]:
EDdata.save_path

'./processed/myhosp/'

#### at any stage save your data out to come back to later..

In [22]:
EDdata.saveRAWasRAW()

----------------------------------------
saved file: ./processed/myhosp/RAW/myhospED.pkl


#### once your data is cleaned, save cleaned data out to .pkl file

In [23]:
EDdata.saveRAWasCLEAN()

----------------------------------------
saved file: ./processed/myhosp/myhospED.pkl


#### load your data back from cleaned file again if you are using ioED:

In [24]:
EDdata.loadPKLasRAW()

----------------------------------------
loaded file: ./processed/myhosp/myhospED.pkl
